In [16]:
import sys
import numpy as np
import impl.RNN as rnn
import impl.solver as solver

In [17]:
# if __name__ == '__main__':
with open('data/text_data/japan.txt', 'r') as f:
    txt = f.read()

    X = []
    y = []

    char_to_idx = {char: i for i, char in enumerate(set(txt))}
    idx_to_char = {i: char for i, char in enumerate(set(txt))}

    X = np.array([char_to_idx[x] for x in txt])
    y = [char_to_idx[x] for x in txt[1:]]
    y.append(char_to_idx['.'])
    y = np.array(y)

In [18]:
vocab_size = len(char_to_idx)

# hyper parameters
time_step = 100 # width
num_layers = 4 # depth
n_iter = 13000 # epochs
alpha = 1e-3 # learning_rate
print_after = 1000 # print loss, valid, and test
H = 64 # num_hidden_units in hidden layer

In [19]:
import numpy as np
import impl.loss as loss_fun
import impl.layer as l
import impl.regularization as reg
import impl.utils as util
import impl.NN as nn

class RNN(nn.NN):

    def __init__(self, D, H, L, char2idx, idx2char):
        self.D = D
        self.H = H
        self.L = L
        self.char2idx = char2idx
        self.idx2char = idx2char
        self.vocab_size = len(char2idx)
        super().__init__(D, D, H, None, None, loss='cross_ent', nonlin='relu')

    def _init_model(self, D, C, H):
        self.model = []
        for l in range(self.L):
            m = dict(
                Wxh=np.random.randn(D, H) / np.sqrt(D / 2.),
                Whh=np.random.randn(H, H) / np.sqrt(H / 2.),
                Why=np.random.randn(H, D) / np.sqrt(C / 2.),
                bh=np.zeros((1, H)),
                by=np.zeros((1, D))
                )
            self.model.append(m)
            
    def initial_state(self):
        return np.zeros((1, self.H))

    def forward(self, X, h, m):
        Wxh, Whh, Why = m['Wxh'], m['Whh'], m['Why']
        bh, by = m['bh'], m['by']

        hprev = h.copy()

        h, h_cache = l.tanh_forward(X @ Wxh + hprev @ Whh + bh)
        y, y_cache = l.fc_forward(h, Why, by)

        cache = X, Whh, h, hprev, y, h_cache, y_cache, Wxh

        return y, h, cache

    def backward(self, dy, dh, cache):
        X, Whh, h, hprev, y, h_cache, y_cache, Wxh = cache

        dh_next = dh.copy()
        
        # Hidden to output gradient
        dh, dWhy, dby = l.fc_backward(dy, y_cache)
        dh += dh_next
        dby = dby.reshape((1, -1))

        # tanh
        dh = l.tanh_backward(dh, h_cache)

        # Hidden gradient
        dbh = dh
        dWhh = hprev.T @ dh
        dWxh = X.T @ dh
        
        dX = dh @ Wxh.T
        dh = dh @ Whh.T

        grad = dict(Wxh=dWxh, Whh=dWhh, Why=dWhy, bh=dbh, by=dby)
        
        return dX, dh, grad

    def train_step_fwd(self, X_train, h):
        ys , caches = [], []
        h_init = h.copy()
        h = []
        for l in range(self.L):
            h.append(h_init.copy())
            caches.append([])
            
        for X in X_train:
            X_one_hot = np.zeros(self.D)
            X_one_hot[X] = 1.
            y = X_one_hot.reshape(1, -1)
            for l in range(self.L):
                y, h[l], cache = self.forward(y, h[l], self.model[l])
                caches[l].append(cache)
                
            ys.append(y)
            
        return ys, caches

    def train_step_bwd(self, y_train, ys, caches):
        loss, dys = 0.0, []

        for y_pred, y in zip(ys, y_train):
            loss += loss_fun.cross_entropy(self.model[0], y_pred, y, lam=0)/ y_train.shape[0]
            dy = loss_fun.dcross_entropy(y_pred, y)
            dys.append(dy)

        # Grads
        dh, grad, grads = [], [], []
        for l in range(self.L):
            dh.append(np.zeros((1, self.H)))
            grad.append({key: np.zeros_like(val) for key, val in self.model[0].items()})
            grads.append({key: np.zeros_like(val) for key, val in self.model[0].items()})
            
        for t in reversed(range(len(dys))):
            dX = dys[t]
            for l in reversed(range(self.L)):
                dX, dh[l], grad[l] = self.backward(dX, dh[l], caches[l][t])
                for k in grad[0].keys():
                    grads[l][k] += grad[l][k]
                
        return loss, grads
    
    def test(self, X_seed, h, size=100):
        chars = [self.idx2char[X_seed]]
        idx_list = list(range(self.vocab_size))
        X = X_seed
        
        h_init = h.copy()
        h = []
        for l in range(self.L):
            h.append(h_init.copy())

        for _ in range(size - 1):
            X_one_hot = np.zeros(self.D)
            X_one_hot[X] = 1.
            y = X_one_hot.reshape(1, -1)
            for l in range(self.L):
                y, h[l], _ = self.forward(y, h[l], self.model[l])
                
            prob = util.softmax(y)
            idx = np.random.choice(idx_list, p=prob.ravel())
            chars.append(self.idx2char[idx])
            X = idx

        return ''.join(chars)

In [20]:
net = RNN(D=vocab_size, H=H, L=num_layers, char2idx=char_to_idx, idx2char=idx_to_char)

In [21]:
import numpy as np
import impl.utils as util
import impl.constant as c
import copy
from sklearn.utils import shuffle as skshuffle

def get_minibatch(X, y, minibatch_size, shuffle=True):
    minibatches = []

    if shuffle:
        X, y = skshuffle(X, y)

    for i in range(0, X.shape[0], minibatch_size):
        X_mini = X[i:i + minibatch_size]
        y_mini = y[i:i + minibatch_size]

        minibatches.append((X_mini, y_mini))

    return minibatches

def adam_rnn(nn, X_train, y_train, alpha=0.001, mb_size=256, n_iter=2000, print_after=100):
    minibatches = get_minibatch(X_train, y_train, mb_size, shuffle=False)

    idx = 0
    state = nn.initial_state()
    smooth_loss = -np.log(1.0 / len(set(X_train)))
    
    M, R = [], []
    for l in range(nn.L):
        M.append({k: np.zeros_like(v) for k, v in nn.model[0].items()})
        R.append({k: np.zeros_like(v) for k, v in nn.model[0].items()})
        
    beta1 = .9
    beta2 = .999

    for iter in range(1, n_iter + 1):
        t = iter

        if idx >= len(minibatches):
            idx = 0
            state = nn.initial_state()

        X_mini, y_mini = minibatches[idx]
        idx += 1

        # Print loss and test sample
        if iter % print_after == 0:
            print('Iter-{} loss: {:.4f}'.format(iter, smooth_loss))
            sample = nn.test(X_mini[0], state)
            print(sample)

        ys, caches = nn.train_step_fwd(X_mini, state)
        loss, grads = nn.train_step_bwd(y_mini, ys, caches)
        smooth_loss = 0.999 * smooth_loss + 0.001 * loss

        for l in range(nn.L):
            for k in grads[0].keys(): #key, value: items
                M[l][k] = util.exp_running_avg(M[l][k], grads[l][k], beta1)
                R[l][k] = util.exp_running_avg(R[l][k], grads[l][k]**2, beta2)

                m_k_hat = M[l][k] / (1. - beta1**(t))
                r_k_hat = R[l][k] / (1. - beta2**(t))

                nn.model[l][k] -= alpha * m_k_hat / (np.sqrt(r_k_hat) + c.eps)
    
    return nn

In [22]:
adam_rnn(nn=net, X_train=X, y_train=y, alpha=alpha, mb_size=time_step, n_iter=n_iter, 
                print_after=print_after)

Iter-1000 loss: 3.4263
JS1nostm mteredr, jurlhns 1al fat 2ars klathy Jesydns 日 anattrs ansd tfes 1mliAnh  eara tlaikr comh5
Iter-2000 loss: 2.5178
 –avindhe . thebe mpirs Gg Warda anreary ede curind ro mthd A puth lhe GeccofircoaltiEght eeettcCfd 
Iter-3000 loss: 1.7415
 Paleolirand Hukal cimutury oonexd8t winthise fn Hsuthc rovo, warang.uwaran Seweltir Ersy af rhe fin
Iter-4000 loss: 1.1195
ery thes inith is the salinat of 1bo ond Sh the eaplca os a.min hereceess arloworytthu mcititile Nap
Iter-5000 loss: 0.6742
 PapeoT7,h on pron-walcnifs corsan whm icothin, woned loma and Ifstory world's fod of lalany. fopoot
Iter-6000 loss: 1.6446
 Aopotim, the here, of Japan wan iJonal Jape the Firs bye far in lroput det,t Am imh anke cenpese 3a
Iter-7000 loss: 1.4995
n the wou omthy The owserg frob cofn an his permesth lar eromimeslires acitian Euperan tornl divviag
Iter-8000 loss: 1.5131
. The Mopuko2 wonld's gys and borlth Japan Sias peacmo anded dir heukop tint bared its sakes and sou
Iter-900